In [1]:
'''
--------------------Objective of the notebook--------------------
In this notebook, let us explore the given dataset and make some inferences along the way. 
Also finally we will build a LSTM model.



--------------------Goal of the project--------------------
In this project, our goal is to predict customers' spending during the period 2018.12 - 2019.1 given 
data of 2018.5 - 2018.10 (test_v2.csv)



--------------------Our strategy--------------------
our strategy is to set the data from May 1st 2017 to October 31st 2017 as the 
training set and use the data from December 1st 2017 to January 31st 2018 for the target values.



--------------------File Descriptions--------------------
train_v2.csv - the updated training set - contains user transactions from August 1st 2016 to April 30th 2018.

test_v2.csv - the updated test set - contains user transactions from May 1st 2018 to October 15th 2018.

sample_submission_v2.csv - a updated sample submission file in the correct format. 
Contains all fullVisitorIds in test_v2.csv. Your submission's PredictedLogRevenue column 
should make forward-looking predictions for each of these fullVisitorIds for the timeframe 
of December 1st 2018 to January 31st 2019.



'''

"\n---------Objective of the notebook----------\nIn this notebook, let us explore the given dataset and make some inferences along the way. \nAlso finally we will build a LSTM model.\n\n\n\n---------Goal of the project---------\nIn this project, our goal is to predict customers' spending during the period 2018.12 - 2019.1 given \ndata of 2018.5 - 2018.10 (test_v2.csv)\n\n\n\n---------Our strategy---------\nour strategy is to set the data from May 1st 2017 to October 31st 2017 as the \ntraining set and use the data from December 1st 2017 to January 31st 2018 for the target values.\n\n\n\n---------File Descriptions---------\ntrain_v2.csv - the updated training set - contains user transactions from August 1st 2016 to April 30th 2018.\n\ntest_v2.csv - the updated test set - contains user transactions from May 1st 2018 to October 15th 2018.\n\nsample_submission_v2.csv - a updated sample submission file in the correct format. \nContains all fullVisitorIds in test_v2.csv. Your submission's Pr

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import time
from datetime import datetime
from pandas import json_normalize
#from pandas.io.json import json_normalize
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from zipfile import ZipFile
gc.enable()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/home/username/input/train_v2.csv.zip'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
'''
-------------In the data loading process, we only choose data from May 1st 2017 to January 31st 2018---------
'''


def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['geoNetwork','device','totals','trafficSource']
    JSON_ARR_COL = ['customDimensions']
    zf = ZipFile(csv_path)
    frames = [tf[(20180201 > tf['date']) & (tf['date'] >= 20170501)] for tf in pd.read_csv(zf.open('train_v2.csv'),
                                       converters={column: json.loads for column in JSON_COLUMNS},
                                       dtype={'fullVisitorId': 'str'}, # Important!!
                                       chunksize = 50000)]
    
    df = pd.concat(frames,ignore_index=True)
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

    return df

In [4]:
X_train = load_df('/Users/hyunkyujun/train_v2.csv.zip')


In [5]:
'''
--------------------There are 748307 rows--------------------
'''

X_train.shape 

(748307, 59)

In [6]:
pd.set_option('display.max_columns', 5000)

In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748307 entries, 0 to 748306
Data columns (total 59 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   channelGrouping                                    748307 non-null  object
 1   customDimensions                                   748307 non-null  object
 2   date                                               748307 non-null  int64 
 3   fullVisitorId                                      748307 non-null  object
 4   hits                                               748307 non-null  object
 5   socialEngagementType                               748307 non-null  object
 6   visitId                                            748307 non-null  int64 
 7   visitNumber                                        748307 non-null  int64 
 8   visitStartTime                                     748307 non-null  int64 
 9   geoN

In [8]:
'''
--------------------Data Type Observation--------------------
As we can see, most columns are categorical columns.
'''

X_train.head()

,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.timeOnSite,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,3162355547410993243,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508198450,1,1508198450,Europe,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,Firefox,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,1,1,1,1,1,1,NaN,NaN,NaN,NaN,(not set),google,organic,water bottle,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,8934116514970143966,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1508176307,6,1508176307,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Cupertino,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,Chrome,not available in demo dataset,not available in demo dataset,Chrome OS,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,1,2,2,NaN,NaN,2,28,NaN,NaN,NaN,(not set),sites.google.com,referral,NaN,not available in demo dataset,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,7992466427990357681,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508201613,1,1508201613,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,windjammercable.net,not available in demo dataset,not available in demo dataset,not available in demo dataset,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo datas

In [9]:
for c in X_train.columns:
    print('Column Name:' + c, 'Number of Unique Value:'+ str(X_train[c].nunique()))

8
6
276
575250


KeyboardInterrupt: 

In [4]:
df = X_train.copy()

In [6]:
'''
--------------------Data Type change--------------------

Change the type of columns 'totals.transactions', 'totals.transactionRevenue' and 
'totals.totalTransactionRevenue' as float and replace the empty value with '0'.
'''

transaction_col = ['totals.transactions','totals.transactionRevenue' ,'totals.totalTransactionRevenue']
for c in transaction_col:
    df[c] = df[c].astype(float)
    df[c] = df[c].fillna(0)

In [7]:
'''
--------------------Visit date type--------------------
During the training, we will utilize the timing of each visits monthly-wise. 
For instance, for a visit of 23rd, September 2017, the 'date_' will be '20170901'
'''

import datetime
df['date_'] = df['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]),1))
cnt_srs = df.groupby('date_')['date_'].agg(['count'])
cnt_srs.columns = ["count"]
x = range(cnt_srs.shape[0])


In [8]:
'''
As we mentioned earlier, we use customers' log data 
from May 1st 2017 to October 31st 2017 as the training set
'''

train = df.loc[df['date'] < 20171101]

In [9]:
'''
--------------------Columns with constant values--------------------
The columns with constant value will be dropped to save some memory and time in our modeling process.
'''

train = train.drop(['hits','customDimensions'],axis=1)
train = train.drop([c for c in train.columns if train[c].nunique(dropna=False)==1], axis=1)
train = train.drop([c for c in train.columns if train[c].nunique(dropna=False)==0], axis=1)

In [204]:
import plotly
import plotly.subplots
import plotly.offline as py
from plotly import tools
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go



In [205]:
def horizontal_bar_chart(cnt_srs, color):
    trace = go.Bar(
        y=cnt_srs.index[::-1],
        x=cnt_srs.values[::-1],
        showlegend=False,
        orientation = 'h',
        )
    return trace



In [206]:
'''
--------------------trafficSource Information--------------------
Here, we explore how traffic sources are related to average transactions for each visit.
Throughout this observation, we'd like to determine proper number of dimension of traffic source
columns during Principle Component Analysis (PCA) algorithm.

'''


traffic_source = ['trafficSource.campaign', 'trafficSource.source',
       'trafficSource.medium', 'trafficSource.keyword',
       'trafficSource.referralPath', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.isTrueDirect']

i = 1
traffic_source_dict = {}
for c in traffic_source:
    cnt_srs = train.groupby(c)['totals.transactionRevenue'].agg(['size', 'mean'])
    cnt_srs.columns = ["count", "mean"]
    cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
    traffic_source_dict[(i,1)] = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(50, 171, 96, 0.6)')
    traffic_source_dict[(i,2)] = horizontal_bar_chart(cnt_srs["mean"].head(10), 'rgba(50, 171, 96, 0.6)')
    i += 1

In [207]:



subplot_titles =[]
for c in traffic_source:
    subplot_titles.append(c+'_COUNT')
    subplot_titles.append(c+'_MEAN')
    
fig = plotly.subplots.make_subplots(rows=len(traffic_source), cols=2,
                                    subplot_titles=subplot_titles)
for i in range(len(traffic_source)):
    fig.append_trace(traffic_source_dict[(i+1,1)], i+1, 1)
    fig.append_trace(traffic_source_dict[(i+1,2)], i+1, 2)
fig['layout'].update(height=3000, width=1000, paper_bgcolor='rgb(233,233,233)', title="Device Plots")
py.iplot(fig, filename='device-plots')

In [208]:
for c in traffic_source:
    print(c, 'number of unique:'+ str(train[c].nunique()), 'number of null:' +str(train[c].isnull().sum()/train.shape[0]))

trafficSource.campaign number of unique:21 number of null:0.0
trafficSource.source number of unique:146 number of null:0.0
trafficSource.medium number of unique:7 number of null:0.0
trafficSource.keyword number of unique:884 number of null:0.5329295116916517
trafficSource.referralPath number of unique:798 number of null:0.757436296572028
trafficSource.adContent number of unique:47 number of null:0.9405231633447261
trafficSource.adwordsClickInfo.page number of unique:4 number of null:0.928733376197083
trafficSource.adwordsClickInfo.slot number of unique:2 number of null:0.928733376197083
trafficSource.adwordsClickInfo.gclId number of unique:9638 number of null:0.9286995363947075
trafficSource.adwordsClickInfo.adNetworkType number of unique:2 number of null:0.928733376197083
trafficSource.adwordsClickInfo.isVideoAd number of unique:1 number of null:0.928733376197083
trafficSource.isTrueDirect number of unique:1 number of null:0.6710094413048627


In [10]:
pca_dim_traffic = {'trafficSource.campaign':6,'trafficSource.source': 4, 'trafficSource.medium': 5,
                  'trafficSource.keyword': 4, 'trafficSource.referralPath':3,'trafficSource.adContent':2,
                  'trafficSource.adwordsClickInfo.page':2, 'trafficSource.adwordsClickInfo.slot':3,
                  'trafficSource.adwordsClickInfo.gclId': 2, 'trafficSource.adwordsClickInfo.adNetworkType':3,
                  'trafficSource.adwordsClickInfo.isVideoAd':2, 'trafficSource.isTrueDirect':2
                  }
pca_dim_traffic = {'trafficSource.campaign':6,'trafficSource.source': 4, 'trafficSource.medium': 5,
                  'trafficSource.adContent':2,
                  'trafficSource.adwordsClickInfo.page':2, 'trafficSource.adwordsClickInfo.slot':3,
                   'trafficSource.adwordsClickInfo.adNetworkType':3,
                  'trafficSource.adwordsClickInfo.isVideoAd':2, 'trafficSource.isTrueDirect':2
                  }


In [211]:
geoNetwork = ['geoNetwork.continent', 'geoNetwork.subContinent',
       'geoNetwork.country', 'geoNetwork.region', 'geoNetwork.metro',
       'geoNetwork.city', 'geoNetwork.networkDomain']


i = 1
geoNetwork_dict = {}
for c in geoNetwork:
    cnt_srs = train.groupby(c)['totals.transactionRevenue'].agg(['size', 'mean'])
    cnt_srs.columns = ["count", "mean"]
    cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
    geoNetwork_dict[(i,1)] = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(50, 171, 96, 0.6)')
    geoNetwork_dict[(i,2)] = horizontal_bar_chart(cnt_srs["mean"].head(10), 'rgba(50, 171, 96, 0.6)')
    i += 1


In [212]:
subplot_titles =[]
for c in geoNetwork:
    subplot_titles.append(c+'_COUNT')
    subplot_titles.append(c+'_MEAN')

fig = plotly.subplots.make_subplots(rows=len(geoNetwork), cols=2,subplot_titles=subplot_titles)
for i in range(len(geoNetwork)):
    fig.append_trace(geoNetwork_dict[(i+1,1)], i+1, 1)
    fig.append_trace(geoNetwork_dict[(i+1,2)], i+1, 2)
fig['layout'].update(height=2000, width=1000, paper_bgcolor='rgb(233,233,233)', title="Device Plots")
py.iplot(fig, filename='device-plots')

In [11]:
pca_dim_geoNetwork = {'geoNetwork.continent': 6,'geoNetwork.subContinent': 10, 'geoNetwork.country':10,
                     'geoNetwork.region':10, 'geoNetwork.metro':10, 'geoNetwork.city':10, 
                     'geoNetwork.networkDomain':8} 
pca_dim_geoNetwork = {'geoNetwork.continent': 6,'geoNetwork.subContinent': 10, 'geoNetwork.country':10,
                     'geoNetwork.region':10, 'geoNetwork.metro':10
                     } 

In [214]:
for c in geoNetwork:
    print(c, 'number of unique:'+ str(train[c].nunique()), 'number of null:' +str(train[c].isnull().sum()/train.shape[0]))

geoNetwork.continent number of unique:6 number of null:0.0
geoNetwork.subContinent number of unique:23 number of null:0.0
geoNetwork.country number of unique:202 number of null:0.0
geoNetwork.region number of unique:281 number of null:0.0
geoNetwork.metro number of unique:91 number of null:0.0
geoNetwork.city number of unique:523 number of null:0.0
geoNetwork.networkDomain number of unique:9986 number of null:0.0


In [215]:
device = ['device.browser',
       'device.operatingSystem', 'device.isMobile', 'device.deviceCategory']
    
i = 1
device_dict = {}
for c in device:
    cnt_srs = train.groupby(c)['totals.transactionRevenue'].agg(['size', 'mean'])
    cnt_srs.columns = ["count", "mean"]
    cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
    device_dict[(i,1)] = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(50, 171, 96, 0.6)')
    device_dict[(i,2)] = horizontal_bar_chart(cnt_srs["mean"].head(10), 'rgba(50, 171, 96, 0.6)')
    i += 1
    

In [216]:
subplot_titles =[]
for c in device:
    subplot_titles.append(c+'_COUNT')
    subplot_titles.append(c+'_MEAN')

fig = plotly.subplots.make_subplots(rows=len(device), cols=2,subplot_titles=subplot_titles)
for i in range(len(device)):
    fig.append_trace(device_dict[(i+1,1)], i+1, 1)
    fig.append_trace(device_dict[(i+1,2)], i+1, 2)
fig['layout'].update(height=1200, width=1000, paper_bgcolor='rgb(233,233,233)', title="Device Plots")
py.iplot(fig, filename='device-plots')

In [12]:
pca_dim_device = {'device.browser':8, 'device.operatingSystem':8,
                 'device.isMobile':2, 'device.deviceCategory':3} 

In [218]:
for c in device:
    print(c, 'number of unique:'+ str(train[c].nunique()), 'number of null:' +str(train[c].isnull().sum()/train.shape[0]))

device.browser number of unique:38 number of null:0.0
device.operatingSystem number of unique:18 number of null:0.0
device.isMobile number of unique:2 number of null:0.0
device.deviceCategory number of unique:3 number of null:0.0


In [13]:
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler



def one_hot_pca(df,column_name,dummy_na,n_comp = None):
    # return Numpy array
    y = pd.get_dummies(df[column_name], prefix=column_name[:5],dummy_na=dummy_na)
    sc = StandardScaler()
    y = pd.DataFrame(sc.fit_transform(y),columns = y.columns,index=y.index)
    if n_comp:
        pca = PCA(n_components = n_comp)
    else:
        pca = PCA(n_components = 'mle')
    y = pd.DataFrame(pca.fit_transform(y), index=y.index,columns=[column_name + str(i) for i in range(n_comp)])
    return y
    
    

In [14]:
def one_hot_pca_batch(df,column_name, dummy_na, batch_size,n_comp = None):
    # return Numpy array
    y = pd.get_dummies(df[column_name], prefix=column_name[:5],dummy_na=dummy_na)
    sc = StandardScaler()
    y = pd.DataFrame(sc.fit_transform(y),columns = y.columns,index=y.index)
    if n_comp:
        n_bachtes = batch_size
        inc_pca = IncrementalPCA(n_components = n_comp)
        for x_batch in np.array_split(y,n_bachtes):
            inc_pca.partial_fit(x_batch)
    else:
        pca = PCA(n_components = 'mle')
    y = pd.DataFrame(inc_pca.transform(y), index=y.index,columns=[column_name + str(i) for i in range(n_comp)])
    return y

In [15]:
def one_hot(df,column_name,dummy_na):
    # return Numpy array
    y = pd.get_dummies(df[column_name], prefix=column_name[:5],dummy_na=dummy_na)
    sc = StandardScaler()
    y = pd.DataFrame(sc.fit_transform(y),columns = y.columns,index=y.index)
    return y
    

In [16]:
def scale(df,column_name):
    # return Numpy array
    #df[column_name] = df[column_name].astype(float)
    sc = StandardScaler()
    df[column_name] = sc.fit_transform(sc.fit_transform(df[[column_name]]))
    

In [17]:
for k,v in pca_dim_device.items():
    y = one_hot_pca(train,k,True,v)
    train = train.drop(k,axis=1)
    train = train.join(y)

In [18]:
for k,v in pca_dim_geoNetwork.items():
    y = one_hot_pca(train,k,True,v)
    train = train.drop(k,axis=1)
    train = train.join(y)

In [19]:
for k,v in pca_dim_traffic.items():
    y = one_hot_pca(train,k,True,v)
    train = train.drop(k,axis=1)
    train = train.join(y)

In [20]:
train = train.drop(['trafficSource.referralPath','trafficSource.keyword',
                    'trafficSource.adwordsClickInfo.gclId',
                   'geoNetwork.city','geoNetwork.networkDomain',
                   'visitStartTime','visitId',
                   'date'],axis=1)

In [21]:
to_be_one_hot = ['channelGrouping','date_']

for c in to_be_one_hot:
    y = one_hot(train,c,train[c].isnull().sum() > 1)
    train = train.drop(c,axis=1)
    train = train.join(y)

In [22]:
train['totals.newVisits'] = train['totals.newVisits'].astype(float)
train['totals.newVisits'].fillna(0, inplace=True)

In [23]:
train['totals.timeOnSite'] = train['totals.timeOnSite'].astype(float)
train['totals.timeOnSite'].fillna(train['totals.timeOnSite'].mean(), inplace=True)

In [24]:
train['totals.bounces'] = train['totals.bounces'].astype(float)
train['totals.bounces'].fillna(0, inplace=True)

In [25]:
train['totals.hits'] = train['totals.hits'].astype(float)
train['totals.pageviews'] = train['totals.pageviews'].astype(float)
train['totals.pageviews'].fillna(0, inplace=True)


In [26]:
train = train.drop(['totals.sessionQualityDim'],axis=1)

In [27]:
to_be_scaled = ['visitNumber','totals.hits','totals.pageviews','totals.timeOnSite',
                'totals.newVisits',
                'totals.transactions',
                'totals.transactionRevenue','totals.totalTransactionRevenue','totals.bounces']
for c in to_be_scaled:
    scale(train,c)

In [59]:
train.info(verbose=True,max_cols=5000)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 474124 entries, 0 to 748306
Data columns (total 120 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   fullVisitorId                                  474124 non-null  object 
 1   visitNumber                                    474124 non-null  float64
 2   totals.hits                                    474124 non-null  float64
 3   totals.pageviews                               474124 non-null  float64
 4   totals.bounces                                 474124 non-null  float64
 5   totals.newVisits                               474124 non-null  float64
 6   totals.timeOnSite                              474124 non-null  float64
 7   totals.transactions                            474124 non-null  float64
 8   totals.transactionRevenue                      474124 non-null  float64
 9   totals.totalTransactionRevenue      

In [67]:
pd.set_option('display.max_columns', None)

In [72]:
train.columns.tolist()

['fullVisitorId',
 'visitNumber',
 'totals.hits',
 'totals.pageviews',
 'totals.bounces',
 'totals.newVisits',
 'totals.timeOnSite',
 'totals.transactions',
 'totals.transactionRevenue',
 'totals.totalTransactionRevenue',
 'device.browser0',
 'device.browser1',
 'device.browser2',
 'device.browser3',
 'device.browser4',
 'device.browser5',
 'device.browser6',
 'device.browser7',
 'device.operatingSystem0',
 'device.operatingSystem1',
 'device.operatingSystem2',
 'device.operatingSystem3',
 'device.operatingSystem4',
 'device.operatingSystem5',
 'device.operatingSystem6',
 'device.operatingSystem7',
 'device.isMobile0',
 'device.isMobile1',
 'device.deviceCategory0',
 'device.deviceCategory1',
 'device.deviceCategory2',
 'geoNetwork.continent0',
 'geoNetwork.continent1',
 'geoNetwork.continent2',
 'geoNetwork.continent3',
 'geoNetwork.continent4',
 'geoNetwork.continent5',
 'geoNetwork.subContinent0',
 'geoNetwork.subContinent1',
 'geoNetwork.subContinent2',
 'geoNetwork.subContinent3',

In [30]:
#train = train.drop(['trafficSource.campaignCode'],axis=1)

In [28]:
train_dict = train.groupby('fullVisitorId')[train.columns[1:]].apply(lambda g: g.values.tolist()).to_dict()


In [29]:
target = df.loc[(20171201 <= df['date'])  &(df['date'] < 20180201)]
target_dict = target.groupby('fullVisitorId')['totals.transactionRevenue'].apply(lambda g: g.values.sum()).to_dict()

In [30]:
import math
from collections import defaultdict
train_dict2 = defaultdict(list)
target_dict_log = defaultdict(list)
for k,v in train_dict.items():
    temp = np.asarray(v)
    train_dict2[temp.shape[0]].append(temp)
    #target_dict_log[temp.shape[0]].append([target_dict[k]] if k in target_dict else [0.0]) 
    target_dict_log[temp.shape[0]].append([math.log(1+target_dict[k])] if k in target_dict else [0.0] )

In [31]:
for k,v in train_dict2.items():
    train_dict2[k] = np.asarray(v)


In [32]:
for k,v in target_dict_log.items():
    target_dict_log[k] = np.asarray(v)


In [141]:
for k in target_dict_log:
    print(k,target_dict_log[k].mean(),target_dict_log[k].shape[0],train_dict2[k].shape[1] )

3 0.04316583956792523 8034 3
1 0.0026127661728777814 249967 1
2 0.01318886453100398 28031 2
4 0.015498110641498128 3369 4
12 0.4442787552475577 88 12
16 0.0 36 16
9 0.26619114578117825 216 9
8 0.046689596207239135 368 8
7 0.13324070421483863 544 7
6 0.03729526829190694 932 6
11 0.0 110 11
5 0.12201497307709372 1598 5
18 0.7231842070615058 28 18
10 0.23878546772000878 161 10
13 0.2843306583564494 64 13
38 0.0 3 38
20 0.8639841385043361 20 20
35 0.0 4 35
159 0.0 1 159
14 0.43001976832381233 48 14
15 0.0 56 15
19 0.0 19 19
25 0.0 11 25
48 0.0 1 48
33 0.0 5 33
37 0.0 2 37
42 0.0 1 42
44 0.0 1 44
17 0.4888679323841249 36 17
26 0.0 9 26
63 0.0 3 63
27 0.0 3 27
73 0.0 1 73
32 0.0 4 32
23 0.0 16 23
43 0.0 2 43
36 0.0 4 36
24 0.0 11 24
21 0.0 11 21
53 0.0 2 53
40 0.0 4 40
116 9.808345970815793 2 116
30 0.0 4 30
28 0.0 5 28
22 2.034515849989626 9 22
50 0.0 2 50
70 0.0 2 70
46 0.0 1 46
65 0.0 1 65
31 0.0 4 31
59 0.0 1 59
58 0.0 2 58
72 0.0 1 72
29 0.0 5 29
34 0.0 1 34
55 0.0 1 55
49 0.0 1 49
45 5

In [49]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras import optimizers


model = Sequential()

model.add(LSTM(250, return_sequences=True, input_shape=(None, 119)))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation='linear'))
#model.add(TimeDistributed(Dense(1)))
print(model.summary(100))

model.compile(loss='mse',
              optimizer='adam')





Model: "sequential_5"
____________________________________________________________________________________________________
Layer (type)                                 Output Shape                            Param #        
lstm_15 (LSTM)                               (None, None, 250)                       370000         
____________________________________________________________________________________________________
lstm_16 (LSTM)                               (None, None, 100)                       140400         
____________________________________________________________________________________________________
lstm_17 (LSTM)                               (None, 20)                              9680           
____________________________________________________________________________________________________
dense_5 (Dense)                              (None, 1)                               21             
Total params: 520,101
Trainable params: 520,101
Non-trainable params:

In [50]:
for j in range(5):
    for i in range(1,6):
        model.fit(train_dict2[i],target_dict_log[i], epochs=1, 
                  batch_size=max(256//(2**i),16),verbose=1)
#validation_split=0.1,

#model.fit(train_dict2[1],target_dict_log[1], epochs=5, validation_split=0.2,batch_size=128,verbose=1)
#model.fit(train_dict2[2],target_dict_log[2], epochs=10, validation_split=0.1,batch_size=32,verbose=1)
#model.fit(train_dict2[3],target_dict_log[3], epochs=40, validation_split=0.1,batch_size=16,verbose=1)

125/125 [==============================] - 3s 20ms/step - loss: 1.8818


In [52]:
model.save('google_revenue_range_1to6_5times_LSTM.h5')

In [48]:
import keras
model =  keras.models.load_model('google_revenue_10times_LSTM.h5')

In [145]:
for k in test_y_dict:
    print(k,test_y_dict[k].mean(),test_y_dict[k].shape )

3 0.02693467285061493 (2008, 1)
1 0.0031513532817087054 (62491, 1)
2 0.026417665114135985 (7007, 1)
4 0.04100963353480054 (842, 1)
12 0.0 (22, 1)
16 0.0 (8, 1)
9 0.3929958177727032 (53, 1)
8 0.0 (92, 1)
7 0.12360472339143183 (136, 1)
6 0.08573585625346675 (232, 1)
11 0.0 (27, 1)
5 0.04366031279280486 (399, 1)
18 0.0 (6, 1)
10 0.5042800366725164 (40, 1)
13 0.0 (16, 1)
38 nan (0,)
20 0.0 (4, 1)
35 0.0 (1, 1)
159 nan (0,)
14 0.0 (12, 1)
15 0.0 (14, 1)
19 0.0 (4, 1)
25 0.0 (2, 1)
48 nan (0,)
33 0.0 (1, 1)
37 nan (0,)
42 nan (0,)
44 nan (0,)
17 0.0 (8, 1)
26 0.0 (2, 1)
63 nan (0,)
27 nan (0,)
73 nan (0,)
32 0.0 (1, 1)
23 0.0 (3, 1)
43 nan (0,)
36 nan (0,)
24 0.0 (2, 1)
21 0.0 (2, 1)
53 nan (0,)
40 0.0 (1, 1)
116 nan (0,)
30 nan (0,)
28 0.0 (1, 1)
22 0.0 (2, 1)
50 nan (0,)
70 nan (0,)
46 nan (0,)
65 nan (0,)
31 0.0 (1, 1)
59 nan (0,)
58 nan (0,)
72 nan (0,)
29 0.0 (1, 1)
34 nan (0,)
55 nan (0,)
49 nan (0,)
45 nan (0,)
52 nan (0,)
94 nan (0,)
107 nan (0,)
68 nan (0,)
47 nan (0,)
78 nan (0,)
6

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice.
  
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [44]:
model.predict(test_x_dict[1])

array([[0.15295665],
       [0.1806544 ],
       [0.20814055],
       ...,
       [0.1939418 ],
       [0.21696794],
       [0.14512521]], dtype=float32)

In [149]:
sqr_sum = 0
for i in range(1,10):
    y_pred = model.predict(test_x_dict[i])
    sqr_sum += np.sum((y_pred-test_y_dict[i])**2)

In [150]:
total_num = 0
for i in range(1,10):
    total_num += test_x_dict[i].shape[0]

In [151]:
np.sqrt((1/total_num)*sqr_sum)

0.5103313359876351